# Import the Required Libraries

# GANESH REDDY JANGA (SecqurAIse Technologies Private Limited Assignment)

In [115]:
import cv2
import numpy as np
import pandas as pd

In [116]:
# Load the video
cap = cv2.VideoCapture('AI Assignment video.mp4')

In [117]:
# Get the video dimensions
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [118]:
width

1920.0

In [119]:
height

1080.0

In [120]:
# Get the dimensions of the frame
ret, frame = cap.read()
height, width, _ = frame.shape

In [121]:
height

1080

In [122]:
# Convert the frame to grayscale
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [123]:
# Detect the edges in the frame
edges = cv2.Canny(gray, 50, 150)


In [124]:
# Display the edges image
cv2.imshow('Edges', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [125]:
# Find the contours in the edge detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [126]:
# Filter the contours based on their area
contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 500]

In [127]:
# Draw the contours on the frame
cv2.drawContours(frame, contours, -1, (0, 255, 0), 3)

# Display the frame with contours
cv2.imshow('Contours', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [128]:
# Initialize the coordinates of the box
top_left = bottom_right = None


In [129]:
# Find the coordinates of the box
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if top_left is None or x < top_left[0] or y < top_left[1]:
        top_left = (x, y)
    if bottom_right is None or x + w > bottom_right[0] or y + h > bottom_right[1]:
        bottom_right = (x + w, y + h)

In [130]:
# Print the coordinates of the box
print('Top Left:', top_left)
print('Bottom Right:', bottom_right)

Top Left: (794, 19)
Bottom Right: (1744, 508)


In [131]:
# Define the boundaries of the quadrants
quadrants = [
    ((top_left[0], top_left[1]), (top_left[0] + (bottom_right[0] - top_left[0]) // 2, top_left[1] + (bottom_right[1] - top_left[1]) // 2)),  # Quadrant 3
    ((top_left[0] + (bottom_right[0] - top_left[0]) // 2, top_left[1]), (bottom_right[0], top_left[1] + (bottom_right[1] - top_left[1]) // 2)),  # Quadrant 4
    ((top_left[0], top_left[1] + (bottom_right[1] - top_left[1]) // 2), (top_left[0] + (bottom_right[0] - top_left[0]) // 2, bottom_right[1])),  # Quadrant 2
    ((top_left[0] + (bottom_right[0] - top_left[0]) // 2, top_left[1] + (bottom_right[1] - top_left[1]) // 2), (bottom_right[0], bottom_right[1]))  # Quadrant 1
]

In [132]:
# Initialize a dictionary to keep track of each ball's last known quadrant
last_known_quadrant = {}

In [133]:
# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640, 480))

In [134]:
# Define the color ranges for the balls
color_ranges = {
    'yellow': [np.array([20, 100, 100]), np.array([30, 255, 255])],
    'white': [np.array([0, 0, 200]), np.array([180, 20, 255])],
    'orange': [np.array([5, 100, 100]), np.array([15, 255, 255])],
    'dark green': [np.array([60, 100, 100]), np.array([80, 255, 255])]
}


In [135]:
# Define the minimum and maximum acceptable area for the balls
min_area = 1000
max_area = 10000

In [136]:
# Open the text file in write mode
with open('events.txt', 'w') as f:
    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        if not ret:
            break

        # Convert the frame to HSV
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        for color, (lower_range, upper_range) in color_ranges.items():
            # Create a mask for the color
            mask = cv2.inRange(hsv_frame, lower_range, upper_range)

            # Erode the mask to remove noise
            mask = cv2.erode(mask, None, iterations=2)

            # Dilate the mask to fill in gaps
            mask = cv2.dilate(mask, None, iterations=2)

            # Find the contours in the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for contour in contours:
                # Calculate the area of the contour
                area = cv2.contourArea(contour)

                # If the area is within the acceptable range, draw a circle around the contour
                if min_area < area < max_area:
                    # Calculate the center and radius of the minimum enclosing circle
                    center, radius = cv2.minEnclosingCircle(contour)

                    # Draw the circle in the original frame
                    cv2.circle(frame, (int(center[0]), int(center[1])), int(radius), (0, 255, 0), 2)

                    # Check if the ball's center is within any of the quadrants
                    for i, quadrant in enumerate(quadrants):
                        if quadrant[0][0] <= center[0] <= quadrant[1][0] and quadrant[0][1] <= center[1] <= quadrant[1][1]:
                            # If the ball was not in this quadrant in the last frame, record an entry event
                            if last_known_quadrant.get(color) != i:
                                event = f'Time: {cap.get(cv2.CAP_PROP_POS_MSEC) / 1000}, Quadrant: {i + 1}, Color: {color}, Event: Entry'
                                print(event)
                                f.write(event + '\n')
                                last_known_quadrant[color] = i
                            break
                    else:
                        # If the ball was in a quadrant in the last frame but is not in any quadrant now, record an exit event
                        if color in last_known_quadrant:
                            event = f'Time: {cap.get(cv2.CAP_PROP_POS_MSEC) / 1000}, Quadrant: {last_known_quadrant[color] + 1}, Color: {color}, Event: Exit'
                            print(event)
                            f.write(event + '\n')
                            del last_known_quadrant[color]

        # Write the frame into the file 'output.avi'
        out.write(frame)

        # Display the frame
        cv2.imshow('Frame', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the video capture and writer objects
cap.release()
out.release()

# Close all OpenCV windows
cv2.destroyAllWindows()

Time: 0.03327777777777778, Quadrant: 2, Color: white, Event: Entry
Time: 0.03327777777777778, Quadrant: 2, Color: white, Event: Exit
Time: 0.03327777777777778, Quadrant: 1, Color: white, Event: Entry
Time: 0.03327777777777778, Quadrant: 2, Color: white, Event: Entry
Time: 0.03327777777777778, Quadrant: 1, Color: white, Event: Entry
Time: 0.03327777777777778, Quadrant: 2, Color: white, Event: Entry
Time: 0.06655555555555556, Quadrant: 1, Color: white, Event: Entry
Time: 0.06655555555555556, Quadrant: 2, Color: white, Event: Entry
Time: 0.06655555555555556, Quadrant: 1, Color: white, Event: Entry
Time: 0.06655555555555556, Quadrant: 2, Color: white, Event: Entry
Time: 0.09983333333333334, Quadrant: 2, Color: white, Event: Exit
Time: 0.09983333333333334, Quadrant: 2, Color: white, Event: Entry
Time: 0.09983333333333334, Quadrant: 1, Color: white, Event: Entry
Time: 0.09983333333333334, Quadrant: 2, Color: white, Event: Entry
Time: 0.13311111111111112, Quadrant: 2, Color: white, Event: Exi